In [ ]:
import os,sys,time,platform

from typing import List
from bfxapi import Client
from bfxapi.types import (
    DepositAddress,
    LightningNetworkInvoice,
    Notification,
    Transfer,
    Wallet,
    Withdrawal,
)

bfx = Client(api_key=os.getenv("BF_API_KEY"), api_secret=os.getenv("BF_API_SECRET"))  # 初始化Bitfinex客戶端


wallets: List[Wallet] = bfx.rest.auth.get_wallets()

print(wallets)
for wallet in wallets:
    print(wallet)




In [ ]:
def list_lending_offers(currency):
    try:
        return bfx.rest.auth.get_funding_offers(symbol=currency)
    except Exception as e:
        print(f"Error getting lending offers: {e}")
        return []


my_offers = list_lending_offers("fUSD")
print(f"my_offers: {my_offers}")
    
    
    

In [ ]:
import requests

url = f"https://api-pub.bitfinex.com/v2/candles/trade:1h:fUSD:a30:p2:p30/hist"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

In [ ]:
async def submit_order(order):
    try:
        # 實際提交訂單的代碼（目前被註釋掉）
        notification: Notification[FundingOffer] = bfx.rest.auth.submit_funding_offer(
            type="LIMIT", symbol=currency, amount=str(order['amount']), rate=order['rate'], period=order['period']
        )
        return True
    except Exception as e:
        print(f"Error submitting funding offer: {e}")
        return False

In [1]:
import requests
import pandas as pd
import time
from datetime import datetime, timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 獲取 Bitfinex fUSD 利率資料
def get_fusd_rate_data(timeframe="15m", start=None, end=None, limit=1000):
    """使用 candles API 獲取 fUSD 利率"""
    # Bitfinex API 接受的是毫秒時間戳
    url = f"https://api-pub.bitfinex.com/v2/candles/trade:{timeframe}:fUSD:a30:p2:p30/hist"
    
    # 構建請求參數
    params = {"limit": min(limit, 1000)}
    if start: params["start"] = int(start)  # 確保是整數
    if end: params["end"] = int(end)        # 確保是整數
    
    # 添加排序參數，1 表示升序 (從舊到新)，-1 表示降序 (從新到舊)
    # Bitfinex API 預設為降序 (-1)
    params["sort"] = 1  # 請求按時間升序排序
    
    try:
        print(f"請求 fUSD 利率數據，URL: {url}, 參數: {params}")
        response = requests.get(url, headers={"accept": "application/json"}, params=params)
        response.raise_for_status()
        data = response.json()
        
        if isinstance(data, list) and len(data) > 0:
            print(f"收到 {len(data)} 條 fUSD 數據")
            
            # [0]MTS, [1]OPEN, [2]CLOSE, [3]HIGH, [4]LOW, [5]VOLUME
            df = pd.DataFrame(data, columns=['mts', 'open_rate', 'close_rate', 'high_rate', 'low_rate', 'volume'])
            
            # 將時間戳轉換為可讀格式
            df['datetime'] = pd.to_datetime(df['mts'], unit='ms')
            
            # 檢查收到的數據順序
            print(f"fUSD 數據時間範圍: {df['datetime'].min()} 到 {df['datetime'].max()}")
            
            # 按時間排序
            df = df.sort_values('datetime')
            return df
        else:
            print(f"API 返回無效數據: {data}")
            return None
    except Exception as e:
        print(f"獲取 fUSD 利率資料時出錯: {e}")
        return None

# 獲取比特幣價格數據
def get_btc_price_data(timeframe="15m", start=None, end=None, limit=1000):
    """獲取 BTC/USD 價格及波動性資料"""
    url = f"https://api-pub.bitfinex.com/v2/candles/trade:{timeframe}:tBTCUSD/hist"
    
    # 構建請求參數
    params = {"limit": min(limit, 1000)}
    if start: params["start"] = int(start)  # 確保是整數
    if end: params["end"] = int(end)        # 確保是整數
    
    # 添加排序參數，1 表示升序 (從舊到新)
    params["sort"] = 1
    
    try:
        print(f"請求 BTC 價格數據，URL: {url}, 參數: {params}")
        response = requests.get(url, headers={"accept": "application/json"}, params=params)
        response.raise_for_status()
        data = response.json()
        
        if isinstance(data, list) and len(data) > 0:
            print(f"收到 {len(data)} 條 BTC 數據")
            
            df = pd.DataFrame(data, columns=['mts', 'open', 'close', 'high', 'low', 'volume'])
            
            # 將時間戳轉換為可讀格式
            df['datetime'] = pd.to_datetime(df['mts'], unit='ms')
            
            # 檢查收到的數據順序
            print(f"BTC 數據時間範圍: {df['datetime'].min()} 到 {df['datetime'].max()}")
            
            # 按時間排序
            df = df.sort_values('datetime')
            
            # 計算波動性
            df['volatility_percent'] = ((df['high'] - df['low']) / df['low']) * 100
            
            # 移動平均波動性 (使用rolling前確保數據已排序)
            df['volatility_ma24'] = df['volatility_percent'].rolling(window=24).mean()
            
            return df
        else:
            print(f"API 返回無效數據: {data}")
            return None
    except Exception as e:
        print(f"獲取 BTC 價格資料時出錯: {e}")
        return None

# 處理時間範圍，確保獲取足夠資料
def get_time_range(days=7):
    """計算時間範圍，並處理時區問題"""
    end_time = int(time.time() * 1000)  # 當前時間的毫秒時間戳
    start_time = end_time - (days * 24 * 60 * 60 * 1000)  # 前 n 天
    
    # 轉換為可讀格式，用於顯示
    end_datetime = datetime.fromtimestamp(end_time/1000)
    start_datetime = datetime.fromtimestamp(start_time/1000)
    
    print(f"請求時間範圍: {start_datetime} 到 {end_datetime}")
    
    return start_time, end_time

# 分成三張圖但共享時間軸的互動式繪圖函數
def plot_triple_chart(days=7, timeframe="15m"):
    """繪製 fUSD 利率、BTC 價格、BTC 波動性的三張互動式圖表，共享時間軸"""
    # 計算時間範圍
    start_time, end_time = get_time_range(days)
    
    # 獲取資料
    fusd_data = get_fusd_rate_data(timeframe=timeframe, start=start_time, end=end_time, limit=1000)
    # 稍等一下避免API速率限制
    time.sleep(1)
    btc_data = get_btc_price_data(timeframe=timeframe, start=start_time, end=end_time, limit=1000)
    
    # 檢查是否成功獲取數據
    if fusd_data is None or btc_data is None:
        print("無法獲取數據")
        return None, None
    
    if fusd_data.empty or btc_data.empty:
        print("獲取的數據為空")
        return None, None
    
    # 創建時間索引
    time_index = pd.date_range(
        start=min(fusd_data['datetime'].min(), btc_data['datetime'].min()),
        end=max(fusd_data['datetime'].max(), btc_data['datetime'].max()),
        freq=timeframe.replace('m', 'min').replace('h', 'H').replace('D', 'D')
    )
    
    print(f"創建時間索引: {len(time_index)} 個時間點")
    print(f"時間索引範圍: {time_index.min()} 到 {time_index.max()}")
    
    # 使用時間索引重新索引兩個數據框
    fusd_reindex = fusd_data.set_index('datetime').reindex(time_index)
    btc_reindex = btc_data.set_index('datetime').reindex(time_index)
    
    # 合併數據
    merged = pd.DataFrame(index=time_index)
    merged['close_rate'] = fusd_reindex['close_rate']
    merged['close'] = btc_reindex['close']
    merged['volatility_percent'] = btc_reindex['volatility_percent']
    merged['volatility_ma24'] = btc_reindex['volatility_ma24']
    
    # 前向填充缺失值，然後後向填充剩餘缺失值
    merged = merged.ffill().bfill()
    
    # 重設索引，將日期時間變為列
    merged = merged.reset_index()
    merged = merged.rename(columns={'index': 'datetime'})
    
    print(f"合併後數據: {len(merged)} 行")
    print(f"合併後時間範圍: {merged['datetime'].min()} 到 {merged['datetime'].max()}")
    
    # 創建三個子圖 (垂直排列，共享 x 軸)
    fig = make_subplots(rows=3, cols=1, 
                        shared_xaxes=True,
                        vertical_spacing=0.02,
                        subplot_titles=("fUSD 利率 (%)", "BTC 價格 (USD)", "BTC 波動性 (%)"),
                        row_heights=[0.33, 0.33, 0.34])
    
    # 圖表1: fUSD 利率
    fig.add_trace(
        go.Scatter(
            x=merged['datetime'], 
            y=merged['close_rate'],
            name="fUSD 利率",
            line=dict(color="blue", width=2),
            hovertemplate="<b>%{x}</b><br>利率: %{y:.4f}%<extra></extra>"
        ),
        row=1, col=1
    )
    
    # 添加 fUSD 平均利率參考線
    fusd_avg = merged['close_rate'].mean()
    fig.add_shape(
        type="line", line=dict(dash="dash", color="blue", width=1),
        y0=fusd_avg, y1=fusd_avg, x0=merged['datetime'].iloc[0], x1=merged['datetime'].iloc[-1],
        row=1, col=1
    )
    fig.add_annotation(
        text=f"平均: {fusd_avg:.4f}%",
        xref="paper", yref="y",
        x=1.01, y=fusd_avg,
        showarrow=False,
        row=1, col=1
    )
    
    # 圖表2: BTC 價格
    fig.add_trace(
        go.Scatter(
            x=merged['datetime'], 
            y=merged['close'],
            name="BTC 價格",
            line=dict(color="orange", width=2),
            hovertemplate="<b>%{x}</b><br>價格: $%{y:,.0f}<extra></extra>"
        ),
        row=2, col=1
    )
    
    # 添加 BTC 平均價格參考線
    btc_avg = merged['close'].mean()
    fig.add_shape(
        type="line", line=dict(dash="dash", color="orange", width=1),
        y0=btc_avg, y1=btc_avg, x0=merged['datetime'].iloc[0], x1=merged['datetime'].iloc[-1],
        row=2, col=1
    )
    fig.add_annotation(
        text=f"平均: ${btc_avg:,.0f}",
        xref="paper", yref="y2",
        x=1.01, y=btc_avg,
        showarrow=False,
        row=2, col=1
    )
    
    # 圖表3: BTC 波動性
    fig.add_trace(
        go.Scatter(
            x=merged['datetime'], 
            y=merged['volatility_percent'],
            name="BTC 波動性",
            line=dict(color="red", width=2),
            hovertemplate="<b>%{x}</b><br>波動性: %{y:.4f}%<extra></extra>"
        ),
        row=3, col=1
    )
    
    # 添加 BTC 24小時移動平均波動性
    fig.add_trace(
        go.Scatter(
            x=merged['datetime'], 
            y=merged['volatility_ma24'],
            name="24小時平均波動性",
            line=dict(color="purple", width=2),
            hovertemplate="<b>%{x}</b><br>24h平均: %{y:.4f}%<extra></extra>"
        ),
        row=3, col=1
    )
    
    # 添加波動性平均值參考線
    vol_avg = merged['volatility_percent'].mean()
    fig.add_shape(
        type="line", line=dict(dash="dash", color="red", width=1),
        y0=vol_avg, y1=vol_avg, x0=merged['datetime'].iloc[0], x1=merged['datetime'].iloc[-1],
        row=3, col=1
    )
    fig.add_annotation(
        text=f"平均: {vol_avg:.4f}%",
        xref="paper", yref="y3",
        x=1.01, y=vol_avg,
        showarrow=False,
        row=3, col=1
    )
    
    # 設置全局配置
    fig.update_layout(
        title_text=f"Bitfinex fUSD 利率與 BTC 價格及波動性關係 ({timeframe})",
        title_font=dict(size=20),
        hovermode="x unified",  # 顯示所有數據點的工具提示
        plot_bgcolor='white',   # 設置圖表背景色
        height=900,             # 增加高度以容納三張圖表
        legend=dict(
            orientation="h",     # 水平放置圖例
            yanchor="bottom",
            y=1.02,
            xanchor="center",
            x=0.5
        ),
        margin=dict(r=100)      # 增加右邊距以容納註釋
    )
    
    # 配置X軸和Y軸
    fig.update_xaxes(
        showgrid=True, 
        gridwidth=1, 
        gridcolor='lightgrey',
        zeroline=True,
        zerolinewidth=1,
        zerolinecolor='lightgrey'
    )
    
    # 只在最下面的圖表顯示X軸標題
    fig.update_xaxes(
        title_text="日期時間",
        rangeslider_visible=True,  # 添加時間範圍滑塊
        row=3, col=1
    )
    
    # 為每個Y軸添加網格
    fig.update_yaxes(
        showgrid=True, 
        gridwidth=1, 
        gridcolor='lightgrey',
        zeroline=True,
        zerolinewidth=1,
        zerolinecolor='lightgrey'
    )
    
    # 啟用聯動選擇
    fig.update_layout(
        updatemenus=[
            dict(
                type="buttons",
                direction="left",
                buttons=list([
                    dict(
                        args=[{"xaxis.autorange": True, 
                               "yaxis.autorange": True,
                               "yaxis2.autorange": True,
                               "yaxis3.autorange": True}],
                        label="重設視圖",
                        method="relayout"
                    ),
                ]),
                pad={"r": 10, "t": 10},
                showactive=False,
                x=0.1,
                xanchor="left",
                y=1.1,
                yanchor="top"
            ),
        ]
    )
    
    # 顯示圖表
    fig.show()
    
    # 保存為HTML文件，可以在瀏覽器中打開
    file_name = f'fusd_btc_triple_chart_{timeframe}_{datetime.now().strftime("%Y%m%d")}.html'
    fig.write_html(file_name)
    print(f"圖表已保存到 {file_name}")
    
    # 計算相關係數
    correlation = merged[['close_rate', 'close', 'volatility_percent']].corr()
    print("\n相關係數:")
    print(correlation)
    
    return fig, merged

if __name__ == "__main__":
    # 獲取7天的1小時資料
    fig, data = plot_triple_chart(days=7, timeframe="1h")


請求時間範圍: 2025-04-24 22:22:55.504000 到 2025-05-01 22:22:55.504000
請求 fUSD 利率數據，URL: https://api-pub.bitfinex.com/v2/candles/trade:1h:fUSD:a30:p2:p30/hist, 參數: {'limit': 1000, 'start': 1745504575504, 'end': 1746109375504, 'sort': 1}
收到 168 條 fUSD 數據
fUSD 數據時間範圍: 2025-04-24 15:00:00 到 2025-05-01 14:00:00
請求 BTC 價格數據，URL: https://api-pub.bitfinex.com/v2/candles/trade:1h:tBTCUSD/hist, 參數: {'limit': 1000, 'start': 1745504575504, 'end': 1746109375504, 'sort': 1}
收到 168 條 BTC 數據
BTC 數據時間範圍: 2025-04-24 15:00:00 到 2025-05-01 14:00:00
創建時間索引: 168 個時間點
時間索引範圍: 2025-04-24 15:00:00 到 2025-05-01 14:00:00
合併後數據: 168 行
合併後時間範圍: 2025-04-24 15:00:00 到 2025-05-01 14:00:00


C:\Users\kao85\AppData\Local\Temp\ipykernel_99572\3360906326.py:134: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  time_index = pd.date_range(


圖表已保存到 fusd_btc_triple_chart_1h_20250501.html

相關係數:
                    close_rate     close  volatility_percent
close_rate            1.000000  0.010570            0.065424
close                 0.010570  1.000000           -0.015166
volatility_percent    0.065424 -0.015166            1.000000
